Code for the Proposed Fixation Detection Algorithm

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
import os

# Load the image
image_path = 'scanpath_image.png'

# Check if the image file exists
if not os.path.isfile(image_path):
    print(f"Error: The file '{image_path}' does not exist.")
else:
    # Load the image
    image = cv2.imread(image_path)

    # Check if the image was loaded correctly
    if image is None:
        print(f"Error: The image '{image_path}' could not be loaded.")
    else:
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Thresholding to create a binary image
        _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)

        # Find contours which correspond to gaze points
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Extract coordinates of gaze points
        gaze_points = []
        for contour in contours:
            M = cv2.moments(contour)
            if M['m00'] != 0:
                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])
                gaze_points.append((cx, cy))

        gaze_points = np.array(gaze_points)

        # Plot the gaze points on the original image
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.scatter(gaze_points[:, 0], gaze_points[:, 1], color='red')
        plt.title('Detected Gaze Points')
        plt.show()

        # Define a threshold distance for fixation (in pixels)
        fixation_threshold = 20

        # Function to calculate fixations
        def calculate_fixations(points, threshold):
            fixations = []
            current_fixation = [points[0]]

            for point in points[1:]:
                if euclidean(point, current_fixation[-1]) <= threshold:
                    current_fixation.append(point)
                else:
                    fixations.append(np.mean(current_fixation, axis=0))
                    current_fixation = [point]
            
            fixations.append(np.mean(current_fixation, axis=0))
            return np.array(fixations)

        # Calculate fixations
        fixations = calculate_fixations(gaze_points, fixation_threshold)


    
        

        

In [ ]:
Code for the Proposed Character Selection Algorithm

In [ ]:
# Load the dataset
        df = pd.read_csv('keyboard coordinates.csv')

        # Function to check if a fixation is within AOI
        def is_within_aoi(fixation, aoi):
            x, y = fixation
            return (aoi['X_MIN'] <= x <= aoi['X_MAX']) and (aoi['Y_MIN'] <= y <= aoi['Y_MAX'])

        # Calculate fixation frequency for each character
        df['fixation_frequency'] = 0

        for fixation in fixations:
            for index, row in df.iterrows():
                if is_within_aoi(fixation, row):
                    df.at[index, 'fixation_frequency'] += 1
# Print fixation frequencies for each character
print("Fixation frequencies for each character:")
print(df[['Character', 'fixation_frequency']])
# Find the maximum fixation frequency
max_frequency = df['fixation_frequency'].max()

# Filter characters with the maximum fixation frequency
max_frequency_chars = df[df['fixation_frequency'] == max_frequency]

# Calculate average distance of fixations from the center of AOI for characters with maximum fixation frequency
for index, row in max_frequency_chars.iterrows():
    fixations_in_aoi = [fixation for fixation in fixations if is_within_aoi(fixation, row)]
    center_x = (row['X_MIN'] + row['X_MAX']) / 2
    center_y = (row['Y_MIN'] + row['Y_MAX']) / 2
    avg_distance = np.mean([np.sqrt((fix[0] - center_x) ** 2 + (fix[1] - center_y) ** 2) for fix in fixations_in_aoi])
    max_frequency_chars.at[index, 'average_distance'] = avg_distance

# Select the character with the minimum average distance
final_character = max_frequency_chars.loc[max_frequency_chars['average_distance'].idxmin()]

# Print characters with the maximum fixation frequency and their average distances
print("Characters with the highest fixation frequency and their average distances to fixations:")
print(max_frequency_chars[['Character', 'fixation_frequency', 'average_distance']])

# Print the final character
print("\nFinal character:")
print(final_character[['Character', 'fixation_frequency', 'average_distance']])

Code for the Proposed Ensemble Model with Meta-Learner

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Load dataset
file_path = 'normalized_dataset_finall.csv'
data = pd.read_csv(file_path)
X = data.drop(columns=['Image name', 'Character'])
y = data['Character']

# Set desired augmented dataset size
desired_size = 1540

# Enhanced Data Augmentation to achieve desired size
def augment_data_enhanced(X, y, desired_size):
    augmented_X = X.copy()
    augmented_y = y.copy()
    
    current_size = X.shape[0]

    while augmented_X.shape[0] < desired_size:
        # Add Gaussian noise and shifts
        X_augmented = X + np.random.normal(0, 0.01, X.shape)
        X_shifted = X + np.random.uniform(-2, 2, X.shape)
        
        # Scale data randomly
        X_scaled = X * np.random.uniform(0.9, 1.1, X.shape)
        
        # Append augmented data to the original dataset
        augmented_X = np.vstack([augmented_X, X_augmented, X_shifted, X_scaled])
        augmented_y = np.hstack([augmented_y, y, y, y])
        
        # Trim if over the desired size
        if augmented_X.shape[0] > desired_size:
            extra_samples = augmented_X.shape[0] - desired_size
            augmented_X = augmented_X[:-extra_samples]
            augmented_y = augmented_y[:-extra_samples]

    return augmented_X, augmented_y

# Apply enhanced augmentation to the data
X_augmented, y_augmented = augment_data_enhanced(X.values, y.values, desired_size)

# Verify augmented dataset shape
print(f"Augmented Dataset Shape: {X_augmented.shape}, Labels Shape: {y_augmented.shape}")

# Split augmented data
X_train, X_test, y_train, y_test = train_test_split(X_augmented, y_augmented, test_size=0.2, random_state=42)

# Custom ensemble with optimized meta-learner
class OptimizedEnsemble(BaseEstimator, ClassifierMixin):
    def __init__(self):
        # Base learners with initial parameter setups
        self.rf = RandomForestClassifier(n_estimators=500, max_depth=25, min_samples_split=3, random_state=42)
        self.svm = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=10, gamma='scale', random_state=42, probability=True))
        self.nn = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(200, 100), max_iter=500, random_state=42))
        
        # Using Logistic Regression as the meta-learner
        self.meta_learner = LogisticRegression(max_iter=200, random_state=42)

    def fit(self, X, y):
        # Train base models
        self.rf.fit(X, y)
        self.svm.fit(X, y)
        self.nn.fit(X, y)

        # Stack predictions as meta-features
        rf_pred = self.rf.predict_proba(X)
        svm_pred = self.svm.predict_proba(X)
        nn_pred = self.nn.predict_proba(X)
        meta_features = np.hstack([rf_pred, svm_pred, nn_pred])
        
        # Train the meta-learner on stacked predictions
        self.meta_learner.fit(meta_features, y)
        return self

    def predict(self, X):
        # Generate predictions from each model
        rf_pred = self.rf.predict_proba(X)
        svm_pred = self.svm.predict_proba(X)
        nn_pred = self.nn.predict_proba(X)

        # Stack predictions and use meta-learner to predict final output
        meta_features = np.hstack([rf_pred, svm_pred, nn_pred])
        return self.meta_learner.predict(meta_features)

    def predict_proba(self, X):
        rf_pred = self.rf.predict_proba(X)
        svm_pred = self.svm.predict_proba(X)
        nn_pred = self.nn.predict_proba(X)
        meta_features = np.hstack([rf_pred, svm_pred, nn_pred])
        return self.meta_learner.predict_proba(meta_features)

# Train the optimized ensemble model
optimized_ensemble_model = OptimizedEnsemble()
optimized_ensemble_model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = optimized_ensemble_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Optimized Ensemble Model Accuracy: {accuracy * 100:.2f}%")
